In [17]:
# --- Erste Zelle ---
import os
import sys
if os.getcwd().endswith('notebooks'):
    os.chdir('..')
sys.path.insert(0, os.getcwd())
print(f"Aktuelles Arbeitsverzeichnis: {os.getcwd()}")

# --- Danach alle Imports ---
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from utils.dataloader import HockeyDataset
from utils.transforms import train_transform, val_transform
from utils.resnet34_lstm import ResNet34_LSTM

from sklearn.metrics import classification_report, f1_score, confusion_matrix


Aktuelles Arbeitsverzeichnis: c:\Users\hp\OneDrive\Desktop\DBU\wai81-ai-theory\ml_picture_recognition


Arbeitverzeichnis

In [18]:
current_dir = os.getcwd()
if current_dir.endswith('notebooks'):
    os.chdir('..')
sys.path.insert(0, os.getcwd())
print(f"Aktuelles Arbeitsverzeichnis: {os.getcwd()}")

Aktuelles Arbeitsverzeichnis: c:\Users\hp\OneDrive\Desktop\DBU\wai81-ai-theory\ml_picture_recognition


Parameter


In [19]:
# --- Pfade und Parameter ---
TRAIN_CSV = 'data/labels_train.csv'
VAL_CSV = 'data/labels_val.csv'
FRAMES_ROOT = 'data/train_frames'  # Hier liegen deine Frames!

In [20]:
BATCH_SIZE = 8
NUM_WORKERS = 4
NUM_CLASSES = 4              # 4 Labels: Check, Neutral, Schuss, Tor
HIDDEN_DIM = 256
LEARNING_RATE = 1e-4
EPOCHS = 30

In [21]:
# --- Dataset & DataLoader ---
train_dataset = HockeyDataset(csv_file=TRAIN_CSV, frames_root=FRAMES_ROOT, transform=train_transform)
val_dataset = HockeyDataset(csv_file=VAL_CSV, frames_root=FRAMES_ROOT, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)


In [22]:
# --- Modell ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet34_LSTM(hidden_dim=HIDDEN_DIM, num_classes=NUM_CLASSES).to(device)

# --- Loss & Optimizer ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [23]:
# --- Vorbereitungen ---
best_val_f1 = 0
early_stopping_counter = 0
patience = 5

train_losses = []
train_f1_scores = []
val_losses = []
val_f1_scores = []

# --- Training Loop ---
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    all_preds, all_targets = [], []

    for frames, labels in train_loader:
        frames = frames.to(device)                     # (B, T, 3, 224, 224)
        labels = labels.float().to(device)              # (B, 4)

        outputs = model(frames)                         # (B, 4)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        all_preds.append((torch.sigmoid(outputs) > 0.5).cpu())
        all_targets.append(labels.cpu())

    train_loss = running_loss / len(train_loader)
    train_f1 = f1_score(torch.cat(all_targets), torch.cat(all_preds), average="macro", zero_division=0)

    train_losses.append(train_loss)
    train_f1_scores.append(train_f1)

    # --- Validation ---
    model.eval()
    val_loss = 0.0
    val_preds, val_targets = [], []

    with torch.no_grad():
        for frames, labels in val_loader:
            frames = frames.to(device)
            labels = labels.float().to(device)

            outputs = model(frames)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            val_preds.append((torch.sigmoid(outputs) > 0.5).cpu())
            val_targets.append(labels.cpu())

    val_loss /= len(val_loader)
    val_f1 = f1_score(torch.cat(val_targets), torch.cat(val_preds), average="macro", zero_division=0)

    val_losses.append(val_loss)
    val_f1_scores.append(val_f1)

    # --- Fortschritt ausgeben ---
    print(f"Epoch {epoch+1}: Train Loss {train_loss:.4f}, F1 {train_f1:.2f} | Val Loss {val_loss:.4f}, F1 {val_f1:.2f}")

    # --- Early Stopping ---
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        early_stopping_counter = 0
        torch.save(model.state_dict(), 'models/best_resnet34_lstm.pth')  # Bester Checkpoint
    else:
        early_stopping_counter += 1
        print(f"Early Stopping Counter: {early_stopping_counter}/{patience}")
        if early_stopping_counter >= patience:
            print("⏹️ Early stopping triggered.")
            break

Epoch 1: Train Loss 1.6232, F1 0.51 | Val Loss 1.4103, F1 0.60
Epoch 2: Train Loss 1.2128, F1 0.74 | Val Loss 1.3292, F1 0.67
Epoch 3: Train Loss 1.0216, F1 0.79 | Val Loss 1.2543, F1 0.73
Epoch 4: Train Loss 0.8104, F1 0.85 | Val Loss 1.5766, F1 0.62
Early Stopping Counter: 1/5
Epoch 5: Train Loss 0.7670, F1 0.86 | Val Loss 1.2374, F1 0.68
Early Stopping Counter: 2/5
Epoch 6: Train Loss 0.7757, F1 0.86 | Val Loss 1.4990, F1 0.75
Epoch 7: Train Loss 0.8465, F1 0.84 | Val Loss 1.2271, F1 0.70
Early Stopping Counter: 1/5


KeyboardInterrupt: 

In [ ]:
# # --- Save Model ---
# torch.save(model.state_dict(), 'models/resnet34_lstm.pth')